In [13]:
import sys

sys.path.append("../")

In [14]:
import requests
import time
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from stats_arb.tests import adf_test, kpss_test, cal_half_life, pp_test
from datetime import datetime, timedelta
from ta.volatility import BollingerBands

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.ioff()

In [15]:
API_BASE = 'https://fapi.binance.com/fapi/v1/'
# DATA_PATH = '/mnt/c/Users/vmodg/source/repos/BeanRepos/crypto-pair-trading/data/'
DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto/'
TIMEFRAME = '1d'
lookback = 365

# TIMEFRAME = '1h'
# lookback = 30*4

# TIMEFRAME = '5m'
# lookback = 20


In [16]:


LABELS = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

DROP_COLUMNS=[
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]


def get_batch(symbol, interval='1m', start_time=0, limit=400):
    """Use a GET request to retrieve a batch of candlesticks. Process the JSON into a pandas
    dataframe and return it. If not successful, return an empty dataframe.
    """

    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': limit
    }
    try:
        # timeout should also be given as a parameter to the function
        response = requests.get(f'{API_BASE}klines', params, timeout=30)
    except requests.exceptions.ConnectionError:
        print('Connection error, Cooling down for 5 mins...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    except requests.exceptions.Timeout:
        print('Timeout, Cooling down for 5 min...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    if response.status_code == 200:
        return pd.DataFrame(response.json(), columns=LABELS)
    
    print(f'Got erroneous response back {symbol}: {response}. {response.text}')
    return pd.DataFrame([])


def get_candles(base, quote, start_date: datetime, end_date=None, interval='1m'):
    batches = []

    if end_date is None:
        end_date = datetime.utcnow()

    last_timestamp = int(start_date.timestamp()) * 1000
    # gather all candlesticks available, starting from the last timestamp loaded from disk or 0
    # stop if the timestamp that comes back from the api is the same as the last one
    previous_timestamp = None

    while previous_timestamp != last_timestamp:
        # stop if we reached data from today
        if datetime.fromtimestamp(last_timestamp / 1000) >= end_date:
            break

        previous_timestamp = last_timestamp

        new_batch = get_batch(
            symbol=base + quote,
            interval=interval,
            start_time=last_timestamp
        )

        # requesting candles from the future returns empty
        # also stop in case response code was not 200
        if new_batch.empty:
            break

        last_timestamp = new_batch['open_time'].max()

        # sometimes no new trades took place yet on date.today();
        # in this case the batch is nothing new
        if previous_timestamp == last_timestamp:
            break

        batches.append(new_batch)
        last_datetime = datetime.fromtimestamp(last_timestamp / 1000)

        covering_spaces = 20 * ' '
        print(datetime.now(), base, quote, interval, str(last_datetime) + covering_spaces, end='\r', flush=True)

    if len(batches) > 0:
        # write clean version of csv to parquet
        df = pd.concat(batches, ignore_index=True)
        df.drop(columns=DROP_COLUMNS, inplace=True)
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df.set_index('open_time', inplace=True)
        df = df[~df.index.duplicated(keep='first')]
        return df



In [17]:
# import requests

# url = "https://fapi.binance.com/fapi/v1/exchangeInfo"

# payload={}
# headers = {}

# response = requests.request("GET", url, headers=headers, data=payload).json()

# symbols = [s['symbol'] for s in response['symbols'] if s['contractType'] == 'PERPETUAL' and s['quoteAsset'] == 'USDT']
# # symbols[100:]
# symbols = symbols[:50]
# len(symbols)


In [18]:
# symbols = pd.read_csv(f'/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/symbols.csv')['symbol'].values.tolist()
symbols = ['BTCUSDT',
'ETHUSDT',
'XRPUSDT',
'LTCUSDT',
'ETCUSDT',
'LINKUSDT',
'XLMUSDT',
'ADAUSDT',
'BNBUSDT',
'ATOMUSDT',
'ALGOUSDT',
'DOTUSDT',
'SOLUSDT',
'AVAXUSDT',
'MATICUSDT',
'XMRUSDT',
'NEARUSDT',
'EOSUSDT',
'NEOUSDT',
'UNIUSDT']
symbols = [s.replace('USDT', '') for s in symbols]

data = []
start_time = datetime.utcnow() - timedelta(days=lookback)

for symbol in symbols:
    df = pd.read_csv(f'{DATA_PATH}/{TIMEFRAME}/{symbol}-USDT.csv', parse_dates=['open_time'], index_col=['open_time'])
    df = df[df.index >= start_time].copy()
    # print('start time', df.index[-1].to_pydatetime())

    df1 = get_candles(
        base=symbol, 
        quote='USDT', 
        start_date=df.index[-1].to_pydatetime(),
        # end_date=datetime.now() - timedelta(days=15),
        interval=TIMEFRAME
    )
    if df1 is not None:
        df = pd.concat([df, df1])
        df = df[~df.index.duplicated(keep='first')]

    df.rename(columns={'close': symbol}, inplace=True)
    # the data is too long, just limit to recent period
    log = np.log(df[symbol].astype(np.float32))
    data.append(log)

df = pd.concat(data, axis=1)
df = df.dropna(axis=1, how='all')
# df.dropna(inplace=True, how='any')

# our of sample
# df = df[df.index <= datetime.now() - timedelta(days=15)].copy()

df.tail()

,BTC,ETH,XRP,LTC,ETC,LINK,XLM,ADA,BNB,ATOM,ALGO,DOT,SOL,AVAX,MATIC,XMR,NEAR,EOS,NEO,UNI
open_time,,,,,,,,,,,,,,,,,,,,
2023-05-06,10.269443,7.547127,-0.781105,4.421488,2.941223,1.933838,-2.382494,-0.971539,5.773775,2.382874,-1.742969,1.727932,3.086852,2.807714,-0.025318,5.050177,0.568151,-0.017146,2.303285,1.664683
2023-05-07,10.254827,7.533426,-0.801401,4.423288,2.929699,1.932535,-2.388579,-0.978432,5.771348,2.395073,-1.753886,1.727576,3.075421,2.797403,-0.037494,5.054907,0.552159,-0.031491,2.289804,1.659941
2023-05-08,10.227736,7.521069,-0.848866,4.352469,2.904275,1.883883,-2.415194,-1.007584,5.747959,2.359344,-1.808499,1.679896,3.026019,2.734108,-0.105583,5.032788,0.495306,-0.098716,2.210579,1.602010
2023-05-09,10.225941,7.520668,-0.847698,4.380526,2.930393,1.874568,-2.419456,-1.013628,5.742715,2.338724,-1.803630,1.671097,3.027618,2.728180,-0.132960,5.042715,0.482426,-0.085558,2.224732,1.599792
2023-05-10,10.228140,7.519980,-0.838635,4.379774,2.920470,1.873186,-2.421143,-1.010327,5.745979,2.336310,-1.801204,1.672788,3.029022,2.728637,-0.133989,5.044135,0.479954,-0.084469,2.246650,1.600801


In [19]:
p = 1
COINTEGRATION_CONFIDENCE_LEVEL = 90

# the 90%, 95%, and 99% confidence levels for the trace statistic and maximum 
# eigenvalue statistic are stored in the first, second, and third column of 
# cvt and cvm, respectively
confidence_level_cols = {
    90: 0,
    95: 1,
    99: 2
}
confidence_level_col = confidence_level_cols[COINTEGRATION_CONFIDENCE_LEVEL]


def test_johansen(symbol_pairs):
    df_t = df[symbol_pairs].copy()
    df_t.dropna(inplace=True)

    # The second and third parameters indicate constant term, with a lag of 1. 
    result = coint_johansen(df_t, 0, p)

    trace_crit_value = result.cvt[:, confidence_level_col]
    eigen_crit_value = result.cvm[:, confidence_level_col]
#     print("trace_crit_value",trace_crit_value)
#     print("eigen_crit_value",eigen_crit_value)
#     print("lr1",result.lr1)
#     print("lr2",result.lr2)

    # The trace statistic and maximum eigenvalue statistic are stored in lr1 and lr2;
    # see if they exceeded the confidence threshold
    if np.all(result.lr1 >= trace_crit_value) and np.all(result.lr2 >= eigen_crit_value):
        # print(f"{symbol_pairs} are cointegrated")
        # The first i.e. leftmost column of eigenvectors matrix, result.evec, contains the best weights.
        v1= result.evec[:,0:1]
        hr=v1/-v1[1] #to get the hedge ratio divide the best_eigenvector by the negative of the second component of best_eigenvector
        #the regression will be: close of symbList[1] = hr[0]*close of symbList[0] + error
        #where the beta of the regression is hr[0], also known as the hedge ratio, and
        #the error of the regression is the mean reverting residual signal that you need to predict, it is also known as the "spread"
        #the spread = close of symbList[1] - hr[0]*close of symbList[0] or alternatively (the same thing):
        #do a regression with close of symbList[0] as x and lose of symbList[1] as y, and take the residuals of the regression to be the spread.
        coint_pair = dict(hedge_ratio=v1[:, 0])
        for i, s in enumerate(symbol_pairs):
            coint_pair[f'sid_{i+1}'] = s

        cointegrating_pairs.append(coint_pair)



In [20]:
import traceback


def calculate_spread(df, coint_df, selected_row, hedge_ratio, nb_symbols=2):
    spread = None
    for i in range(nb_symbols):
        if spread is None:
            spread = df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
        else:
            spread += df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
    
    spread.dropna(inplace=True)
    return spread


critical_val = 0.02

def find_stationary_portfolio(coint_df):
    data = []
    for i, _ in coint_df.iterrows():
        try:
            hedge_ratio = coint_df.iloc[i]['hedge_ratio']
            _df = df.copy()
            spread = calculate_spread(_df, coint_df, i, hedge_ratio)
            p_val = adf_test(spread, verbose=False)
            if p_val < critical_val:
                half_life = cal_half_life(spread)
                pairs_name = coint_df[[col for col in coint_df.columns if 'sid' in col]].iloc[i].values
                # print(i, pairs_name, 'is stationary with half life', half_life)
                # print(' ')
                data.append({
                    'i': i,
                    'pairs': pairs_name,
                    'half_life': half_life
                })
        except:
            print(coint_df.iloc[i])
            # traceback.print_exc()
    
    return pd.DataFrame(data)


# find_stationary_portfolio(coint_df)

In [21]:

import itertools as it

nb_symbols = 2
cointegrating_pairs = []

#get symbol pairs
pairs = list(it.combinations(symbols, nb_symbols))

for pair in pairs:
    try:
        test_johansen(list(pair))
    except Exception:
        pass

coint_df = pd.DataFrame(cointegrating_pairs)
coint_df['i'] = coint_df.index
coint_df.head()

,hedge_ratio,sid_1,sid_2,i
0,"[2.9017308677346954, -11.13468360775328]",BTC,XMR,0
1,"[2.1712553956349083, -12.494557369813885]",ETH,LINK,1
2,"[8.562712826370209, -14.649983700746452]",ETH,BNB,2
3,"[6.825683061127237, -13.987490906615776]",ETH,XMR,3
4,"[1.2336800356286552, 6.106655479577597]",ETH,UNI,4


In [22]:
coint_df.to_csv(f'coint_df_{TIMEFRAME}.csv')


# Result

In [23]:
import warnings

warnings.filterwarnings("ignore")

stationary_df = find_stationary_portfolio(coint_df)
stationary_df = pd.merge(stationary_df, coint_df, on='i')
stationary_df.dropna(inplace=True)
stationary_df['i'] = stationary_df.i.astype('int64')
stationary_df['half_life'] = stationary_df.half_life.astype('int64')
if len(stationary_df) > 0:
    stationary_df.sort_values(by=['half_life'], inplace=True)
# stationary_df


In [24]:
stationary_df.to_csv(f'stationary_df_{TIMEFRAME}.csv', index=False)
stationary_df

,i,pairs,half_life,hedge_ratio,sid_1,sid_2
7,7,"[ETC, LINK]",5,"[2.28542288860416, -13.319195464385649]",ETC,LINK
13,13,"[LINK, SOL]",6,"[11.879867070460254, -0.7703399783597572]",LINK,SOL
18,18,"[LINK, EOS]",6,"[12.868243267608475, -3.400829756724175]",LINK,EOS
17,17,"[LINK, NEAR]",6,"[11.839409711339986, -0.784392026998947]",LINK,NEAR
16,16,"[LINK, XMR]",6,"[10.85284331394359, -0.49828620838305515]",LINK,XMR
15,15,"[LINK, MATIC]",6,"[10.61869307017327, 0.01799005263709258]",LINK,MATIC
14,14,"[LINK, AVAX]",6,"[12.746632978278457, -1.9171436133387043]",LINK,AVAX
12,12,"[LINK, DOT]",6,"[12.233479021613382, -1.864706168317126]",LINK,DOT
11,11,"[LINK, ATOM]",6,"[11.34779097598752, -0.7862842874191698]",LINK,ATOM
19,19,"[LINK, NEO]",6,"[12.70966145250258, -2.4210068285684647]",LINK,NEO
